# DX 704 Week 6 Project

This project will develop a treatment plan for a fictious illness "Twizzleflu".
Twizzleflu is a mild illness caused by a virus.
The main symptoms are a mild fever, fidgeting, and kicking the blankets off the bed or couch.
Mild dehydration has also been reported in more severe cases.
These symptoms typically last 1-2 weeks without treatment.
Word on the internet says that Twizzleflu can be cured faster by drinking copious orange juice, but this has not been supported by evidence so far.
You will be provided with a theoretical model of Twizzleflu modeled as a Markov decision process.
Based on the model, you will compute optimal treatment plans to optimize different criteria, and compare patient discomfort with the different plans.

The full project description, a template notebook, and raw data are available on GitHub: [Project 6 Materials](https://github.com/bu-cds-dx704/dx704-project-06).

We will model Twizzleflu as a Markov decision process.
The model transition probabilities are provided in the file "twizzleflu-transitions.tsv" and the expected rewards are in "twizzleflu-rewards.tsv".
The goal for Twizzleflu is to minimize the expected discomfort of the patient which is expressed as negative rewards in the file.

## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

In [1]:
import pandas as pd

rewards_df = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")
transitions_df = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")

def build_lookups(rewards_table, transitions_table):
    all_states = set(rewards_table["state"].tolist())
    all_states.update(transitions_table["state"].tolist())
    all_states.update(transitions_table["next_state"].tolist())

    all_actions = sorted(set(rewards_table["action"].tolist()) | set(transitions_table["action"].tolist()))

    reward_by_state_action = {}
    for _, row in rewards_table.iterrows():
        reward_by_state_action[(row["state"], row["action"])] = float(row["reward"])

    transitions_by_state_action = {}
    for _, row in transitions_table.iterrows():
        key = (row["state"], row["action"])
        transitions_by_state_action.setdefault(key, []).append((row["next_state"], float(row["probability"])))

    return all_states, all_actions, reward_by_state_action, transitions_by_state_action


def get_available_actions(state, all_actions, reward_by_state_action, transitions_by_state_action):
    available = []
    for action in all_actions:
        if (state, action) in reward_by_state_action or (state, action) in transitions_by_state_action:
            available.append(action)
    return available


def run_value_iteration_optimal(all_states, all_actions, reward_by_state_action, transitions_by_state_action):
    value_by_state = {state: 0.0 for state in all_states}

    max_iterations = 200000
    tolerance = 0.0000000001

    for _ in range(max_iterations):
        biggest_change = 0.0
        new_value_by_state = {}

        for state in all_states:
            available_actions = get_available_actions(state, all_actions, reward_by_state_action, transitions_by_state_action)

            if len(available_actions) == 0:
                best_value = 0.0
            else:
                best_value = None
                for action in available_actions:
                    immediate_reward = float(reward_by_state_action.get((state, action), 0.0))

                    expected_future_value = 0.0
                    for next_state, probability in transitions_by_state_action.get((state, action), []):
                        expected_future_value += probability * value_by_state[next_state]

                    q_value = immediate_reward + expected_future_value

                    if best_value is None or q_value > best_value:
                        best_value = q_value

            new_value_by_state[state] = float(best_value)

            change_amount = abs(new_value_by_state[state] - value_by_state[state])
            if change_amount > biggest_change:
                biggest_change = change_amount

        value_by_state = new_value_by_state

        if biggest_change < tolerance:
            break

    return value_by_state


def get_greedy_policy(all_states, all_actions, value_by_state, reward_by_state_action, transitions_by_state_action):
    best_action_by_state = {}

    for state in all_states:
        available_actions = get_available_actions(state, all_actions, reward_by_state_action, transitions_by_state_action)

        if len(available_actions) == 0:
            best_action_by_state[state] = ""
            continue

        best_action = None
        best_q_value = None

        for action in available_actions:
            immediate_reward = float(reward_by_state_action.get((state, action), 0.0))

            expected_future_value = 0.0
            for next_state, probability in transitions_by_state_action.get((state, action), []):
                expected_future_value += probability * value_by_state[next_state]

            q_value = immediate_reward + expected_future_value

            if best_q_value is None or q_value > best_q_value or (q_value == best_q_value and action < best_action):
                best_q_value = q_value
                best_action = action

        best_action_by_state[state] = best_action

    return best_action_by_state


def run_value_iteration_fixed_policy(all_states, policy_action_by_state, reward_by_state_action, transitions_by_state_action):
    value_by_state = {state: 0.0 for state in all_states}

    max_iterations = 200000
    tolerance = 0.0000000001

    for _ in range(max_iterations):
        biggest_change = 0.0
        new_value_by_state = {}

        for state in all_states:
            action = policy_action_by_state.get(state, "")

            immediate_reward = float(reward_by_state_action.get((state, action), 0.0))

            expected_future_value = 0.0
            for next_state, probability in transitions_by_state_action.get((state, action), []):
                expected_future_value += probability * value_by_state[next_state]

            updated_value = immediate_reward + expected_future_value
            new_value_by_state[state] = updated_value

            change_amount = abs(updated_value - value_by_state[state])
            if change_amount > biggest_change:
                biggest_change = change_amount

        value_by_state = new_value_by_state

        if biggest_change < tolerance:
            break

    return value_by_state


## Part 1: Evaluate a Do Nothing Plan

One of the treatment actions is to do nothing.
Calculate the expected discomfort (not rewards) of a policy that always does nothing.

Hint: for this value calculation and later ones, use value iteration.
The analytical solution has difficulties in practice when there is no discount factor.

Save the expected discomfort by state to a file "do-nothing-discomfort.tsv" with columns state and expected_discomfort.

Submit "do-nothing-discomfort.tsv" in Gradescope.

In [2]:
all_states, all_actions, reward_by_state_action, transitions_by_state_action = build_lookups(rewards_df, transitions_df)

do_nothing_policy = {}
for state in all_states:
    available_actions = get_available_actions(state, all_actions, reward_by_state_action, transitions_by_state_action)
    do_nothing_policy[state] = "do-nothing" if "do-nothing" in available_actions else ""

do_nothing_values = run_value_iteration_fixed_policy(
    all_states,
    do_nothing_policy,
    reward_by_state_action,
    transitions_by_state_action
)

rows = []
for state in sorted(all_states):
    rows.append({"state": state, "expected_discomfort": -float(do_nothing_values[state])})

do_nothing_discomfort_df = pd.DataFrame(rows)
do_nothing_discomfort_df.to_csv("do-nothing-discomfort.tsv", sep="\t", index=False)

print(do_nothing_discomfort_df.head())


        state  expected_discomfort
0   exposed-1             3.413333
1   exposed-2             4.266667
2   exposed-3             5.333333
3   recovered            -0.000000
4  symptoms-1             6.666667


## Part 2: Compute an Optimal Treatment Plan

Compute an optimal treatment plan for Twizzleflu.
It should minimize the expected discomfort (maximize the rewards).

Save the optimal actions for each state to a file "minimum-discomfort-actions.tsv" with columns state and action.

Submit "minimum-discomfort-actions.tsv" in Gradescope.

In [3]:
all_states, all_actions, reward_by_state_action, transitions_by_state_action = build_lookups(rewards_df, transitions_df)

optimal_values = run_value_iteration_optimal(
    all_states,
    all_actions,
    reward_by_state_action,
    transitions_by_state_action
)

optimal_policy = get_greedy_policy(
    all_states,
    all_actions,
    optimal_values,
    reward_by_state_action,
    transitions_by_state_action
)

rows = []
for state in sorted(all_states):
    rows.append({"state": state, "action": optimal_policy[state]})

minimum_discomfort_actions_df = pd.DataFrame(rows)
minimum_discomfort_actions_df.to_csv("minimum-discomfort-actions.tsv", sep="\t", index=False)

print(minimum_discomfort_actions_df.head())


        state      action
0   exposed-1     sleep-8
1   exposed-2     sleep-8
2   exposed-3     sleep-8
3   recovered  do-nothing
4  symptoms-1    drink-oj


## Part 3: Expected Discomfort

Using your previous optimal policy, compute the expected discomfort for each state.

Save your results in a file "minimum-discomfort-values.tsv" with columns state and expected_discomfort.

In [4]:
all_states, all_actions, reward_by_state_action, transitions_by_state_action = build_lookups(rewards_df, transitions_df)

actions_df = pd.read_csv("minimum-discomfort-actions.tsv", sep="\t")
policy_action_by_state = dict(zip(actions_df["state"], actions_df["action"]))

policy_values = run_value_iteration_fixed_policy(
    all_states,
    policy_action_by_state,
    reward_by_state_action,
    transitions_by_state_action
)

rows = []
for state in sorted(all_states):
    rows.append({"state": state, "expected_discomfort": -float(policy_values[state])})

minimum_discomfort_values_df = pd.DataFrame(rows)
minimum_discomfort_values_df.to_csv("minimum-discomfort-values.tsv", sep="\t", index=False)

print(minimum_discomfort_values_df.head())

        state  expected_discomfort
0   exposed-1                 0.75
1   exposed-2                 1.50
2   exposed-3                 3.00
3   recovered                -0.00
4  symptoms-1                 6.00


Submit "minimum-discomfort-values.tsv" in Gradescope.

## Part 4: Minimizing Twizzleflu Duration

Modifiy the Markov decision process to minimize the days until the Twizzle flu is over.
To do so, change the reward function to always be -1 if the current state corresponds to being sick (must have symptoms, exposed does not count) and 0 otherwise.
To be clear, the action does not matter for this reward function.


Save your new reward function in a file "duration-rewards.tsv" in the same format as "twizzleflu-rewards.tsv".

In [5]:
duration_rewards_df = rewards_df.copy()
duration_rewards_df["reward"] = duration_rewards_df["state"].apply(
    lambda state: -1.0 if "symptoms" in str(state) else 0.0
)

duration_rewards_df.to_csv("duration-rewards.tsv", sep="\t", index=False)

print(duration_rewards_df.head())

       action       state  reward
0  do-nothing   exposed-1     0.0
1  do-nothing   exposed-2     0.0
2  do-nothing   exposed-3     0.0
3  do-nothing  symptoms-1    -1.0
4  do-nothing  symptoms-2    -1.0


Submit "duration-rewards.tsv" in Gradescope.

## Part 5: Optimize for Shorter Twizzleflu

Compute an optimal policy to minimize the duration of Twizzleflu.

Save the optimal actions for each state to a file "minimum-duration-actions.tsv" with columns state and action.

In [6]:
duration_rewards_df = pd.read_csv("duration-rewards.tsv", sep="\t")

all_states, all_actions, reward_by_state_action, transitions_by_state_action = build_lookups(duration_rewards_df, transitions_df)

optimal_duration_values = run_value_iteration_optimal(
    all_states,
    all_actions,
    reward_by_state_action,
    transitions_by_state_action
)

optimal_duration_policy = get_greedy_policy(
    all_states,
    all_actions,
    optimal_duration_values,
    reward_by_state_action,
    transitions_by_state_action
)

rows = []
for state in sorted(all_states):
    rows.append({"state": state, "action": optimal_duration_policy[state]})

minimum_duration_actions_df = pd.DataFrame(rows)
minimum_duration_actions_df.to_csv("minimum-duration-actions.tsv", sep="\t", index=False)

print(minimum_duration_actions_df.head())

        state      action
0   exposed-1     sleep-8
1   exposed-2     sleep-8
2   exposed-3     sleep-8
3   recovered  do-nothing
4  symptoms-1  do-nothing


Submit "minimum-duration-actions.tsv" in Gradescope.

## Part 6: Shorter Twizzleflu?

Compute the expected number of days sick for each state to a file.

Save the expected sick days for each state to a file "minimum-duration-days.tsv" with columns state and expected_sick_days.

In [7]:
duration_rewards_df = pd.read_csv("duration-rewards.tsv", sep="\t")
actions_df = pd.read_csv("minimum-duration-actions.tsv", sep="\t")

policy_action_by_state = dict(zip(actions_df["state"], actions_df["action"]))

all_states, all_actions, reward_by_state_action, transitions_by_state_action = build_lookups(duration_rewards_df, transitions_df)

policy_values = run_value_iteration_fixed_policy(
    all_states,
    policy_action_by_state,
    reward_by_state_action,
    transitions_by_state_action
)

rows = []
for state in sorted(all_states):
    rows.append({"state": state, "expected_sick_days": -float(policy_values[state])})

minimum_duration_days_df = pd.DataFrame(rows)
minimum_duration_days_df.to_csv("minimum-duration-days.tsv", sep="\t", index=False)

print(minimum_duration_days_df.head())

        state  expected_sick_days
0   exposed-1                1.25
1   exposed-2                2.50
2   exposed-3                5.00
3   recovered               -0.00
4  symptoms-1               10.00


Submit "minimum-duration-days.tsv" in Gradescope.

## Part 7: Speed vs Pampering

Compute the expected discomfort using the policy to minimize days sick, and compare the results to the expected discomfort when optimizing to minimize discomfort.

Save the results to a file "policy-comparison.tsv" with columns state, speed_discomfort, and minimize_discomfort.

In [8]:
actions_speed_df = pd.read_csv("minimum-duration-actions.tsv", sep="\t")
actions_min_discomfort_df = pd.read_csv("minimum-discomfort-actions.tsv", sep="\t")

speed_policy = dict(zip(actions_speed_df["state"], actions_speed_df["action"]))
min_discomfort_policy = dict(zip(actions_min_discomfort_df["state"], actions_min_discomfort_df["action"]))

all_states, all_actions, reward_by_state_action, transitions_by_state_action = build_lookups(rewards_df, transitions_df)

speed_values = run_value_iteration_fixed_policy(
    all_states,
    speed_policy,
    reward_by_state_action,
    transitions_by_state_action
)

min_discomfort_values = run_value_iteration_fixed_policy(
    all_states,
    min_discomfort_policy,
    reward_by_state_action,
    transitions_by_state_action
)

rows = []
for state in sorted(all_states):
    rows.append(
        {
            "state": state,
            "speed_discomfort": -float(speed_values[state]),
            "minimize_discomfort": -float(min_discomfort_values[state]),
        }
    )

policy_comparison_df = pd.DataFrame(rows)
policy_comparison_df.to_csv("policy-comparison.tsv", sep="\t", index=False)

print(policy_comparison_df.head())

        state  speed_discomfort  minimize_discomfort
0   exposed-1          0.833333                 0.75
1   exposed-2          1.666667                 1.50
2   exposed-3          3.333333                 3.00
3   recovered         -0.000000                -0.00
4  symptoms-1          6.666667                 6.00


Submit "policy-comparison.tsv" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.